In [ ]:
import tensorflow as tf
import matplotlib.image as img
import numpy as np
from collections import defaultdict
import collections
import matplotlib.pyplot as plt
import os
import random
from tensorflow import keras
from tensorflow.keras import models
import pandas as pd
import math
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
# Check if GPU is enabled
print(tf.__version__)
print(tf.test.gpu_device_name())

2.8.2



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load both legitmate & DGA domains

In [ ]:
Domain_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/DLDomain_MCLS2.csv')
Domain_data.drop(["Unnamed: 0"], axis=1, inplace=True)
Domain_data.head()

,domain_name,label,domain
0,odqzesdvd.info,nymaim,odqzesdvd
1,mcrwnxdzu.info,nymaim,mcrwnxdzu
2,poxrscvfoy.biz,nymaim,poxrscvfoy
3,dupzwi.info,nymaim,dupzwi
4,pgldibr.com,nymaim,pgldibr


In [ ]:
grouped = Domain_data.groupby('label')
Domain_data = grouped.filter(lambda x: x['label'].count() > 5.)

In [ ]:
Domain_data.domain=Domain_data.domain.astype(str)

In [ ]:
from sklearn.preprocessing import LabelEncoder
X = Domain_data.domain
Y = Domain_data['label']

In [ ]:
#from sklearn.preprocessing import LabelBinarizer
#Y = LabelBinarizer().fit_transform(Y)

In [ ]:
grouped=Domain_data['label'].groupby(Domain_data['label'])
grouped.count()

label
abcbot                27
antavmu               16
bamital              104
banjori           469896
bigviktor           1000
chinad              1000
conficker            492
copperstealer         11
cryptolocker        1000
dircrypt             736
dyre                1000
emotet            500540
enviserv             492
feodo                247
flubot             29999
fobber_v1            298
fobber_v2            299
gameover           12000
gspy                 100
kfos                 124
legit             613032
locky               1158
matsnu               908
murofet             8560
mydoom             10047
necro               2708
necurs              8190
ngioweb             5274
nymaim               333
omexo                 40
padcrypt             168
proslikefan          100
pykspa_v1          44675
pykspa_v2_fake       800
pykspa_v2_real       199
qadars              2000
ramnit             19532
ranbyus            11160
rovnix            179995
shifu              

In [ ]:
le=LabelEncoder()
Y = le.fit_transform(Y)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential

#max_words = num_words
max_len = 50

# Tokenizer
tok = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tok.fit_on_texts(X)
sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences,padding='post',maxlen=max_len)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sequences_matrix, Y, test_size=0.20,random_state=42)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
strategy = {0:16613,1:16613,2:16613, 4:16613, 5:16613,  6:16613,7:16613, 8:16613, 9:16613, 10:16613, 12:16613, 13:16613, 15:16613, 16:16613, 18:16613, 17:16613,19:16613,21:16613, 22:16613,23:16613, 24:16613,25:16613, 26:16613,27:16613,28:16613,29:16613,30:16613,31:16613,33:16613,34:16613,35:16613,37:16613,42:16613,39:16613,40:16613,43:16613,44:16613,46:16613,47:16613, 48:16613, 49:16613,50:16613,51:16613}
#for i in minmum_classes:
ovrsmt = RandomOverSampler(sampling_strategy=strategy)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
udrsmt = RandomUnderSampler(sampling_strategy={38:143527,20:203528,11:193527,3:193529})

In [ ]:
from imblearn.pipeline import Pipeline
steps = [('o', ovrsmt), ('u', udrsmt)]
pipeline = Pipeline(steps=steps)

In [ ]:
X_train, y_train = pipeline.fit_resample(X_train, y_train)

In [ ]:
import pickle
# save the model to disk
filename = '/content/drive/My Drive/Colab Notebooks/MX_train_sm.csv'
pickle.dump(X_train, open(filename, 'wb'))

In [ ]:
filename = '/content/drive/My Drive/Colab Notebooks/MX_test_sm.csv'
pickle.dump(X_test, open(filename, 'wb'))


In [ ]:
filename = '/content/drive/My Drive/Colab Notebooks/My_train_sm.csv'
pickle.dump(y_train, open(filename, 'wb'))

In [ ]:
filename = '/content/drive/My Drive/Colab Notebooks/My_test_sm.csv'
pickle.dump(y_test, open(filename, 'wb'))

In [ ]:
y_train=pd.Series(y_train)
weights = y_train.value_counts()/len(Domain_data)
weights = dict(zip(np.unique(y_train), weights))

In [ ]:
weights

{0: 0.09800869870058826,
 1: 0.09319369055277969,
 2: 0.09319272745484034,
 3: 0.06911527897094911,
 4: 0.03610413400159489,
 5: 0.017259196622222906,
 6: 0.01156536160475231,
 7: 0.010801143389873603,
 8: 0.007999973033257697,
 9: 0.007999973033257697,
 10: 0.007999973033257697,
 11: 0.007999973033257697,
 12: 0.007999973033257697,
 13: 0.007999973033257697,
 14: 0.007999973033257697,
 15: 0.007999973033257697,
 16: 0.007999973033257697,
 17: 0.007999973033257697,
 18: 0.007999973033257697,
 19: 0.007999973033257697,
 20: 0.007999973033257697,
 21: 0.007999973033257697,
 22: 0.007999973033257697,
 23: 0.007999973033257697,
 24: 0.007999973033257697,
 25: 0.007999973033257697,
 26: 0.007999973033257697,
 27: 0.007999973033257697,
 28: 0.007999973033257697,
 29: 0.007999973033257697,
 30: 0.007999973033257697,
 31: 0.007999973033257697,
 32: 0.007999973033257697,
 33: 0.007999973033257697,
 34: 0.007999973033257697,
 35: 0.007999973033257697,
 36: 0.007999973033257697,
 37: 0.0079999730

In [ ]:
from sklearn.utils import compute_class_weight
class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(y_train),
                                        y = y_train                                                    
                                    )
class_weights = dict(zip(np.unique(y_train), class_weights))
class_weights

{0: 1.876782084465826,
 1: 1.876782084465826,
 2: 1.876782084465826,
 3: 0.16110753824610663,
 4: 1.876782084465826,
 5: 1.876782084465826,
 6: 1.876782084465826,
 7: 1.876782084465826,
 8: 1.876782084465826,
 9: 1.876782084465826,
 10: 1.876782084465826,
 11: 0.1611092032079801,
 12: 1.876782084465826,
 13: 1.876782084465826,
 14: 1.298204637100003,
 15: 1.876782084465826,
 16: 1.876782084465826,
 17: 1.876782084465826,
 18: 1.876782084465826,
 19: 1.876782084465826,
 20: 0.15319258661820864,
 21: 1.876782084465826,
 22: 1.876782084465826,
 23: 1.876782084465826,
 24: 1.876782084465826,
 25: 1.876782084465826,
 26: 1.876782084465826,
 27: 1.876782084465826,
 28: 1.876782084465826,
 29: 1.876782084465826,
 30: 1.876782084465826,
 31: 1.876782084465826,
 32: 0.8699249677528743,
 33: 1.876782084465826,
 34: 1.876782084465826,
 35: 1.876782084465826,
 36: 2.001989262182533,
 37: 1.876782084465826,
 38: 0.21723425396776055,
 39: 1.876782084465826,
 40: 1.876782084465826,
 41: 1.39005710072

## Load Model and extract feature from an intermidiate layer

In [ ]:
import pickle
# load the model from disk
filename = '/content/drive/My Drive/Colab Notebooks/MCLS_lstmmodel64,1000x100_2nd.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
import pickle
filename = '/content/drive/My Drive/Colab Notebooks/MX_train_sm.csv'
X_train = pickle.load(open(filename, 'rb'))

In [ ]:
filename = '/content/drive/My Drive/Colab Notebooks/My_train_sm.csv'
y_train = pickle.load(open(filename, 'rb'))

In [ ]:
filename = '/content/drive/My Drive/Colab Notebooks/MX_test_sm.csv'
X_test = pickle.load(open(filename, 'rb'))

In [ ]:
filename = '/content/drive/My Drive/Colab Notebooks/My_test_sm.csv'
y_test = pickle.load(open(filename, 'rb'))

In [ ]:
from tensorflow import keras
layer_name = 'flatten_2'
intermediate_layer_model = keras.Model(inputs=loaded_model.input,
                                 outputs=loaded_model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(X_train)

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import time
start = time.time()
DTmodel = DecisionTreeClassifier(class_weight=weights).fit(intermediate_output, y_train)
end = time.time()
print("It took {} milliseconds to train DT with feature from multiclass LSTM intermidiate layer.\n".format(end-start))

It took 705.5816650390625 milliseconds to train DT with feature from multiclass LSTM intermidiate layer.



In [ ]:
from sklearn.metrics import precision_score, accuracy_score,classification_report
start = time.time()
DTpred = DTmodel.predict(intermediate_layer_model.predict(X_test))
end = time.time()
print(classification_report(y_test, DTpred))
print("It took {} milliseconds to test DT with feature from multiclass LSTM intermidiate layer.\n".format(end-start))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.91      1.00      0.95        10
           1       1.00      1.00      1.00         5
           2       0.93      1.00      0.97        28
           3       1.00      1.00      1.00     94137
           4       0.09      0.14      0.11       210
           5       0.93      0.84      0.89       197
           6       0.02      0.03      0.03        89
           7       0.00      0.00      0.00         1
           8       0.06      0.05      0.05       214
           9       0.10      0.11      0.10       151
          10       0.99      0.99      0.99       218
          11       0.99      0.97      0.98    100071
          12       0.89      0.91      0.90        94
          13       0.39      0.34      0.37        44
          14       0.87      0.85      0.86      5982
          15       0.32      0.33      0.33        51
          16       0.10      0.08      0.09        63
          17       0.98    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print('accuracy on the train set: ', accuracy_score(DTmodel.predict(intermediate_output), y_train))
print('accuracy on the test set: ', accuracy_score(DTpred, y_test))

accuracy on the train set:  0.999997532854666
accuracy on the test set:  0.9457343250017456


In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

etpprd = DTmodel.predict_proba(intermediate_layer_model.predict(X_test))

In [ ]:
auc = metrics.roc_auc_score(y_test, etpprd, multi_class='ovo', average='weighted')
print(auc)

0.8607072974511469


In [ ]:
import pickle
# save the model to disk
filename = '/content/drive/My Drive/Colab Notebooks/MLSTM+DTmodel.sav'
pickle.dump(DTmodel, open(filename, 'wb'))

### Random Forest Classifier

In [ ]:
import time
from sklearn.ensemble import RandomForestClassifier
start = time.time()
RFmodel = RandomForestClassifier().fit(intermediate_output, y_train)
end = time.time()
print("It took {} milliseconds to train RF with feature from multiclass LSTM intermidiate layer.\n".format(end-start))

It took 3928.7498059272766 milliseconds to train RF with feature from multiclass LSTM intermidiate layer.



In [ ]:
from sklearn.metrics import precision_score, accuracy_score,classification_report
start = time.time()
RFpred = RFmodel.predict(intermediate_layer_model.predict(X_test))
end = time.time()
print("It took {} milliseconds to test LR with feature from multiclass LSTM intermidiate layer.\n".format(end-start))
print(classification_report(y_test, RFpred))

It took 134.22708439826965 milliseconds to test LR with feature from multiclass LSTM intermidiate layer.



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         5
           2       0.88      1.00      0.93        28
           3       1.00      1.00      1.00     94137
           4       0.28      0.19      0.23       210
           5       0.93      0.93      0.93       197
           6       0.00      0.00      0.00        89
           7       0.00      0.00      0.00         1
           8       0.19      0.03      0.05       214
           9       0.28      0.13      0.17       151
          10       1.00      1.00      1.00       218
          11       0.99      1.00      0.99    100071
          12       0.92      0.96      0.94        94
          13       0.48      0.36      0.42        44
          14       0.89      0.98      0.93      5982
          15       0.37      0.37      0.37        51
          16       0.12      0.11      0.12        63
          17       0.99    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print('accuracy on the train set: ', accuracy_score(RFmodel.predict(intermediate_output), y_train))
print('accuracy on the test set: ', accuracy_score(RFpred, y_test))

accuracy on the train set:  0.9999919817776646
accuracy on the test set:  0.9694120536348467


In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

etpprd = RFmodel.predict_proba(intermediate_layer_model.predict(X_test))

In [ ]:
auc = metrics.roc_auc_score(y_test, etpprd, multi_class='ovo', average='weighted')
print(auc)

0.9452120704065385


In [ ]:
import pickle
# save the model to disk
filename = '/content/drive/My Drive/Colab Notebooks/MLSTM+RFmodel.sav'
pickle.dump(RFmodel, open(filename, 'wb'))

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
import time
start = time.time()
Lreg = LogisticRegression().fit(intermediate_output, y_train)
end = time.time()
print("It took {} milliseconds to train LR with feature from multiclass LSTM intermidiate layer.\n".format(end-start))

It took 612.7370665073395 milliseconds to train LR with feature from multiclass LSTM intermidiate layer.



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
from sklearn.metrics import precision_score, accuracy_score,classification_report
start = time.time()
Lregy_pred = Lreg.predict(intermediate_layer_model.predict(X_test))
end = time.time()
print("It took {} milliseconds to test LR with feature from multiclass LSTM intermidiate layer.\n".format(end-start))
print(classification_report(y_test, Lregy_pred))

It took 132.15079164505005 milliseconds to test LR with feature from multiclass LSTM intermidiate layer.

              precision    recall  f1-score   support

           0       0.13      1.00      0.23        10
           1       0.10      1.00      0.19         5
           2       0.77      0.86      0.81        28
           3       1.00      1.00      1.00     94137
           4       0.06      0.40      0.10       210
           5       0.59      0.99      0.74       197
           6       0.02      0.15      0.04        89
           7       0.00      0.00      0.00         1
           8       0.06      0.40      0.11       214
           9       0.08      0.19      0.11       151
          10       0.97      0.99      0.98       218
          11       0.98      0.99      0.99    100071
          12       0.96      0.95      0.95        94
          13       0.11      0.48      0.18        44
          14       0.88      0.90      0.89      5982
          15       0.06      

In [ ]:
print('accuracy on the train set: ', accuracy_score(Lreg.predict(intermediate_output), y_train))
print('accuracy on the test set: ', accuracy_score(Lregy_pred, y_test))

accuracy on the train set:  0.8651427521129558
accuracy on the test set:  0.9408297558309476


In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

etpprd = Lreg.predict_proba(intermediate_layer_model.predict(X_test))

In [ ]:
auc = metrics.roc_auc_score(y_test, etpprd, multi_class='ovo', average='weighted')
print(auc)

0.9874679392508966


In [ ]:
import pickle
# save the model to disk
filename = '/content/drive/My Drive/Colab Notebooks/MLSTM+LRmodel.sav'
pickle.dump(Lreg, open(filename, 'wb'))

### Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
start = time.time()
gnb = GaussianNB().fit(intermediate_output, y_train)
end = time.time()
print("It took {} milliseconds to train NB with feature from multiclass LSTM intermidiate layer.\n".format(end-start))

It took 2.236485242843628 milliseconds to train NB with feature from multiclass LSTM intermidiate layer.



In [ ]:
from sklearn.metrics import precision_score, accuracy_score,classification_report
start = time.time()
gnb_pred = gnb.predict(intermediate_layer_model.predict(X_test))
end = time.time()
print("It took {} milliseconds to test NB with feature from multiclass LSTM intermidiate layer.\n".format(end-start))
print(classification_report(y_test, gnb_pred))

It took 123.98104667663574 milliseconds to test NB with feature from multiclass LSTM intermidiate layer.

              precision    recall  f1-score   support

           0       0.83      1.00      0.91        10
           1       0.00      0.00      0.00         5
           2       0.84      0.75      0.79        28
           3       0.99      0.88      0.93     94137
           4       0.01      0.57      0.02       210
           5       0.75      0.95      0.84       197
           6       0.00      0.11      0.00        89
           7       0.00      0.00      0.00         1
           8       0.06      0.50      0.10       214
           9       0.06      0.17      0.09       151
          10       1.00      1.00      1.00       218
          11       0.99      0.96      0.98    100071
          12       0.94      0.95      0.94        94
          13       0.04      0.61      0.08        44
          14       0.82      0.96      0.89      5982
          15       0.03      

In [ ]:
print('accuracy on the train set: ', accuracy_score(gnb.predict(intermediate_output), y_train))
print('accuracy on the test set: ', accuracy_score(gnb_pred, y_test))

accuracy on the train set:  0.7481877275556079
accuracy on the test set:  0.7559079953867676


In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

etpprd = gnb.predict_proba(intermediate_layer_model.predict(X_test))

In [ ]:
auc = metrics.roc_auc_score(y_test, etpprd, multi_class='ovo', average='weighted')
print(auc)

0.9606169348893484


In [ ]:
import pickle
# save the model to disk
filename = '/content/drive/My Drive/Colab Notebooks/MLSTM+NBmodel.sav'
pickle.dump(gnb, open(filename, 'wb'))

### Adaboost Classifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import time
start = time.time()
ABmodel = AdaBoostClassifier(DecisionTreeClassifier()).fit(intermediate_output, y_train)
end = time.time()
print("It took {} milliseconds to train AB with feature from multiclass LSTM intermidiate layer.\n".format(end-start))

It took 29438.250957727432 milliseconds to train AB with feature from multiclass LSTM intermidiate layer.



In [ ]:
from sklearn.metrics import precision_score, accuracy_score,classification_report
start = time.time()
ABpred = ABmodel.predict(intermediate_layer_model.predict(X_test))
end = time.time()
print(classification_report(y_test, ABpred))
print("It took {} milliseconds to test AB with feature from multiclass LSTM intermidiate layer.\n".format(end-start))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.62      1.00      0.77         5
           2       0.82      1.00      0.90        28
           3       1.00      1.00      1.00     94137
           4       0.28      0.15      0.19       210
           5       0.94      0.88      0.91       197
           6       0.00      0.00      0.00        89
           7       0.00      0.00      0.00         1
           8       0.10      0.02      0.04       214
           9       0.20      0.11      0.14       151
          10       1.00      0.99      0.99       218
          11       0.99      1.00      0.99    100071
          12       0.90      0.97      0.93        94
          13       0.54      0.34      0.42        44
          14       0.89      0.97      0.93      5982
          15       0.32      0.35      0.34        51
          16       0.12      0.08      0.10        63
          17       0.99    

In [ ]:
print('accuracy on the train set: ', accuracy_score(ABmodel.predict(intermediate_output), y_train))
print('accuracy on the test set: ', accuracy_score(ABpred, y_test))

accuracy on the train set:  0.999997532854666
accuracy on the test set:  0.968877535050695


In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

etpprd = ABmodel.predict_proba(intermediate_layer_model.predict(X_test))

In [ ]:
auc = metrics.roc_auc_score(y_test, etpprd, multi_class='ovo', average='weighted')
print(auc)

0.9733394285569053


In [ ]:
import pickle
# save the model to disk
filename = '/content/drive/My Drive/Colab Notebooks/MLSTM+ABmodel.sav'
pickle.dump(ABmodel, open(filename, 'wb'))